<a href="https://colab.research.google.com/github/dthaker05/QueryQuill_ml_backend/blob/main/QueryQuill_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
!pip install fastapi uvicorn spacy nltk scikit-learn sentence-transformers pandas numpy
!pip install pdfplumber
!pip install faiss-cpu sentence-transformers pdfplumber




In [137]:
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [138]:
import spacy
import random

nlp = spacy.load("en_core_web_sm")



In [139]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    pages_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                pages_text.append(text)

    return "\n".join(pages_text)


In [140]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [141]:
import faiss
import numpy as np

def build_faiss_index(text, model):
    sentences = [s.strip() for s in text.split('.') if len(s.strip()) > 30]

    embeddings = model.encode(sentences, show_progress_bar=True)
    embeddings = np.array(embeddings).astype("float32")

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index, sentences


In [142]:
def ingest_new_book(pdf_path, book_metadata):
    print("📘 Ingesting new book:", book_metadata["title"])

    # Step 1: Extract text
    text = extract_text_from_pdf(pdf_path)
    print("✅ Text extracted")

    # Step 2: Build semantic index
    index, sentences = build_faiss_index(text, embedding_model)
    print("✅ Semantic index created")

    book_object = {
        "metadata": book_metadata,
        "full_text": text,
        "faiss_index": index,
        "sentences": sentences
    }

    print("🚀 Book ingestion completed")
    return book_object


In [143]:
pdf_path = list(uploaded.keys())[0]

metadata = {
    "book_id": "OS_001",
    "title": "Operating Systems",
    "subject": "Computer Science"
}

book = ingest_new_book(pdf_path, metadata)


📘 Ingesting new book: Operating Systems
✅ Text extracted


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Semantic index created
🚀 Book ingestion completed


In [144]:
def calculate_retrieval_size(num_questions):
    """
    Decide how much content to fetch so students are well prepared
    """
    BASE_CONTEXT = 40        # minimum sentences for good coverage
    CONTEXT_MULTIPLIER = 2  # scale with number of questions
    MAX_CONTEXT = 120       # upper safety limit

    return min(
        max(BASE_CONTEXT, num_questions * CONTEXT_MULTIPLIER),
        MAX_CONTEXT
    )


In [145]:
def generate_test_from_book(
    book,
    topic,
    difficulty,
    user_requested_questions
):


    # Step 1: Decide how much knowledge to fetch
    retrieval_k = calculate_retrieval_size(num_questions)
    relevant_sentences = retrieve_relevant_content(book, topic, top_k=retrieval_k)
    context = prepare_context(relevant_sentences)

    # Step 4: Generate exactly what user asked for
    questions = questions.append({
    "type": "descriptive",   # 🔹 ADD THIS LINE
    "question": question_text,
    "answer": f"{keyword} explanation based on the given context."
})


    return questions


In [146]:
def prepare_context(relevant_sentences):
    """
    Combines retrieved sentences into a single context string
    """
    return ". ".join(relevant_sentences)


In [147]:
import random

def generate_distractors(correct_answer, keywords, num_distractors=3):
    distractors = set()

    while len(distractors) < num_distractors:
        candidate = random.choice(keywords)
        if candidate.lower() != correct_answer.lower():
            distractors.add(candidate.title())

    return list(distractors)



In [148]:
def generate_mcq(keyword, context_keywords):
    question = f"Which of the following best describes {keyword}?"

    correct_answer = keyword.title()
    distractors = generate_distractors(correct_answer, context_keywords)

    options = distractors + [correct_answer]
    random.shuffle(options)

    return {
        "type": "mcq",
        "question": question,
        "options": options,
        "correct_answer": correct_answer
    }

In [149]:
def extract_keywords(text, num_keywords=10):
    # Assuming nlp (spacy model) is loaded globally
    doc = nlp(text)
    # Filter for meaningful tokens: alphabetic, not stop words, common parts of speech
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop and token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'VERB']]
    # Simple deduplication and selection of top keywords
    unique_keywords = list(dict.fromkeys(keywords))
    return unique_keywords[:num_keywords]

def generate_mcqs(context, num_mcqs=5):
    keywords = extract_keywords(context)

    mcqs = []
    used = set()

    for keyword in keywords:
        # ☀ FILTER WEAK KEYWORDS (THIS IS THE LINE YOU ASKED ABOUT)
        if len(keyword.split()) < 2:
            continue

        if len(mcqs) >= num_mcqs:
            break

        if keyword not in used:
            mcqs.append(generate_mcq(keyword, keywords))
            used.add(keyword)

    return mcqs

In [150]:
def evaluate_mcq(student_answer, correct_answer):
    return 1 if student_answer == correct_answer else 0


In [151]:
from sklearn.metrics.pairwise import cosine_similarity
# Assuming embedding_model is loaded globally

def evaluate_answer(student_answer, correct_answer):
    student_embedding = embedding_model.encode([student_answer])
    correct_embedding = embedding_model.encode([correct_answer])

    similarity = cosine_similarity(student_embedding, correct_embedding)[0][0]
    score = max(0, min(10, int(similarity * 10))) # Scale 0-1 similarity to 0-10 score
    if similarity > 0.8: # Consider high similarity as perfect score for simplicity
        score = 10
    return score, similarity

def evaluate_descriptive(student_answer, correct_answer):
    score, similarity = evaluate_answer(student_answer, correct_answer)

    return {
        "score": score,
        "similarity": round(similarity, 2)
    }

In [152]:
def evaluate_response(question, student_answer):
    if question["type"] == "mcq":
        score = evaluate_mcq(student_answer, question["correct_answer"])
        return {
            "score": score,
            "max_score": 1
        }

    else:
        result = evaluate_descriptive(student_answer, question["answer"])
        return {
            "score": result["score"],
            "similarity": result["similarity"],
            "max_score": 10
        }


In [153]:
def difficulty_split(total_questions):
    return {
        "easy": int(total_questions * 0.4),
        "medium": int(total_questions * 0.4),
        "hard": total_questions - int(total_questions * 0.8)
    }


In [154]:
# Assuming QUESTION_TEMPLATES is a global dictionary
# Assuming extract_keywords is defined
import random # Ensure random is available for shuffling keywords

def generate_questions(text, difficulty, num_questions):
    questions = []
    keywords = extract_keywords(text) # Assumes extract_keywords is defined
    random.shuffle(keywords)

    if not keywords:
        return []

    templates = QUESTION_TEMPLATES.get(difficulty, [])
    if not templates:
        return []

    for i in range(num_questions):
        if not keywords:
            break
        keyword = keywords.pop()

        template = random.choice(templates)
        question_text = template.format(keyword=keyword.title())

        questions.append({
            "type": "descriptive", # Ensure 'type' key is present
            "question": question_text,
            "answer": f"{keyword} explanation based on the given context."
        })
    return questions

def generate_balanced_test(
    book,
    topic,
    total_questions
):
    split = difficulty_split(total_questions)

    retrieval_k = calculate_retrieval_size(total_questions)
    context = prepare_context(
        retrieve_relevant_content(book, topic, retrieval_k)
    )

    test = []

    # EASY (MCQs)
    test.extend(generate_mcqs(context, split["easy"]))

    # MEDIUM (Mixed)
    test.extend(
        generate_questions(
            context,
            difficulty="medium",
            num_questions=split["medium"]
        )
    )

    # HARD (Descriptive)
    test.extend(
        generate_questions(
            context,
            difficulty="hard",
            num_questions=split["hard"]
        )
    )

    return test

In [155]:
topic = "operating systems concepts"
num_questions = 5

retrieval_k = calculate_retrieval_size(num_questions)
relevant_sentences = retrieve_relevant_content(book, topic, top_k=retrieval_k)
context = prepare_context(relevant_sentences)

mcqs = generate_mcqs(context, num_mcqs=num_questions)

for q in mcqs:
    print(q)

test = generate_balanced_test(
    book=book,
    topic="deadlock prevention",
    total_questions=15
)

print("Full test list:")
for q in test:
    print(q)

Full test list:
{'type': 'descriptive', 'question': 'Explain M.', 'answer': 'm explanation based on the given context.'}
{'type': 'descriptive', 'question': 'How does Network work?', 'answer': 'network explanation based on the given context.'}
{'type': 'descriptive', 'question': 'Explain Entire.', 'answer': 'Entire explanation based on the given context.'}
{'type': 'descriptive', 'question': 'Explain High.', 'answer': 'High explanation based on the given context.'}
{'type': 'descriptive', 'question': 'Explain Error.', 'answer': 'error explanation based on the given context.'}
{'type': 'descriptive', 'question': 'How does Thin work?', 'answer': 'thin explanation based on the given context.'}
{'type': 'descriptive', 'question': 'Why is Entire important?', 'answer': 'Entire explanation based on the given context.'}
{'type': 'descriptive', 'question': 'Explain Rate with an example.', 'answer': 'rate explanation based on the given context.'}
{'type': 'descriptive', 'question': 'Explain Dist

In [156]:
student_answers = [
    {"question_id": 0, "answer": "Deadlock prevention"},
    {"question_id": 1, "answer": "Circular wait is avoided"},
    ...
]


In [157]:
def evaluate_test(test_questions, student_answers):
    results = []
    total_score = 0
    max_score = 0

    for idx, question in enumerate(test_questions):
        student_answer = student_answers[idx]["answer"]

        if question["type"] == "mcq":
            score = 1 if student_answer == question["correct_answer"] else 0
            feedback = (
                "Correct answer."
                if score == 1
                else f"Incorrect. Correct answer is {question['correct_answer']}."
            )
            max_q_score = 1

        else:
            score, similarity = evaluate_answer(
                student_answer,
                question["answer"]
            )

            feedback = (
                "Excellent explanation."
                if score >= 8
                else "Good attempt, but missing some points."
                if score >= 5
                else "Answer is incomplete or incorrect."
            )
            max_q_score = 10

        total_score += score
        max_score += max_q_score

        results.append({
            "question": question["question"],
            "student_answer": student_answer,
            "score": score,
            "max_score": max_q_score,
            "feedback": feedback
        })

    return {
        "total_score": total_score,
        "max_score": max_score,
        "percentage": round((total_score / max_score) * 100, 2),
        "detailed_results": results
    }


In [158]:
def overall_feedback(percentage):
    if percentage >= 85:
        return "Excellent performance. Strong conceptual understanding."
    elif percentage >= 60:
        return "Good performance. Some areas need improvement."
    else:
        return "Needs improvement. Revise the topic thoroughly."


In [159]:
def submit_test(test_questions, student_answers):
    evaluation = evaluate_test(test_questions, student_answers)
    summary = overall_feedback(evaluation["percentage"])

    evaluation["summary"] = summary
    return evaluation


In [160]:
# Fake student answers for testing
student_answers = [
    {"answer": q["correct_answer"] if q["type"] == "mcq" else "Deadlock can be prevented by avoiding circular wait"}
    for q in test
]

result = submit_test(test, student_answers)

print(result["percentage"])
print(result["summary"])

3.33
Needs improvement. Revise the topic thoroughly.


In [161]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

app = FastAPI(title="QueryQuill Assessment API")

# -------------------------
# GLOBAL STATE (TEMPORARY)
# -------------------------
current_test = None


# -------- REQUEST MODELS --------
class GenerateTestRequest(BaseModel):
    topic: str
    total_questions: int


class SubmitTestRequest(BaseModel):
    answers: List[str]


# -------- API ENDPOINTS --------
@app.post("/generate-test")
def generate_test_api(data: GenerateTestRequest):
    global current_test

    current_test = generate_balanced_test(
        book=book,  # already ingested book
        topic=data.topic,
        total_questions=data.total_questions
    )

    return {
        "total_questions": len(current_test),
        "questions": current_test
    }


@app.post("/submit-test")
def submit_test_api(data: SubmitTestRequest):
    global current_test

    if current_test is None:
        return {"error": "No active test found. Generate a test first."}

    student_answers = [{"answer": ans} for ans in data.answers]

    result = submit_test(current_test, student_answers)
    return result


@app.get("/health")
def health_check():
    return {"status": "QueryQuill ML API running"}
